# __Graph Game__

#### The Graph Game is a game that revolves around a randomly generated fully connected graph. A player wins by randomly selecting a path that is shorter than the unknown generated distance. <br>

created by Artem, Femi, Tom, Zifan <br>
Github repo: __https://github.com/T0mLam/CS_Summative_Project__
***

# Quickstart Guide

## 1. Download all dependencies

- matplotlib
- networkx
- numpy
- pygame
- scipy 
<br><br>
- random (*)
- tkinter(*)
- unittest (*) <br>

'*' indicates pre-installed python packages

In [ ]:
%pip install matplotlib==3.7.2
%pip install networkx==3.1
%pip install numpy==1.24.3
%pip install pygame==2.5.2
%pip install scipy==1.13.0

## 2. Create all modules

In [2]:
from __future__ import annotations
from typing import List, Tuple, Dict

### __Utilities modules__
These modules contain data structures required for the game.

In [3]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random as rand

#### Node Class
An individual node of the graph data structure. <br><br>
Major contributor: Femi

In [4]:
class Node:
    """Creation of a Node class to represent nodes in the graph data structure."""

    def __init__(self, idx: int) -> None:
        """Initializes a Node object with the parameter value.

        Args:
            idx (int): This are the values associated with the node.
        """
        self.idx = idx
        self.neighbours = {}  # A dictionary to keep the neighbours and their weight

    def __lt__(self, other: type[Node]) -> bool:
        """Compares two nodes based on their values.

        Args:
            other (Node): Another node in the graph to compare with.

        Returns:
            True if the value of the other node is greater than the value of the this node, False otherwise.
        """
        return self.idx < other.idx

    def add_neighbour(self, neighbour: type[Node], weight: int | float) -> None: 
        """Adds a neighbour to the nodes in the graph with an associated weight.

        Args:
            neighbour (Node): Neighbour node to be added.
            weight (int, float): the Weight of the edge connecting the neighbour to the node.
            
        Raises:
            TypeError: If neighbour is not an instance of Node in the graph or if the weight inputed is not numeric (eg" writing a word instead of a number").
        """
        if not isinstance(neighbour, Node):
            raise TypeError("Neighbour must be an instance of Node.")
        if not isinstance(weight, (int, float)):
            raise TypeError("Weight must be a numeric value.")
        
        self.neighbours[neighbour] = weight

    def get_index(self) -> int:
        """A getter method for the idx attribute.

        Returns:
            An integer index of the node.
        """
        return self.idx
    
    def get_neighbours(self) -> List[Tuple[int, int]]:
        """A getter method for the neighbours attribute.

        Returns:
            A List of tuples (idx, wei) consist of the index (idx) and the weight (wei) of each node.
        """
        return self.neighbours.items()

#### Heap Class
A minheap data structure which enables the quick extraction of the minimum element. <br>
It is utilized in the shortest path method and the fizzy search method. <br><br>
Equal contributors: Artem, Femi, Tom, Zifan

In [5]:
class MinHeap:
    """A minheap data structure that enables quick extraction of the minimum element.

    Time Complexity:
        Initialize / heapify O(n * log n)
        Insertion O(log n)
        Extract min O(log n)
        Peak min O(1)
    
    Methods:
        push: Push an element into the minheap.
        pop: Remove and return the smallest element from the minheap.
        top: Return the smallest element from the minheap without removing.
    """

    def __init__(self, arr: List[Tuple[int, type[Node]]] | None = None) -> None:
        """Construct a heap by heapifying the input array.

        Args:
            arr: A list of tuple to be heapified during the initializing stage.
                 For example: [(1, Node_Obj1), (2, Node_Obj2), ...]

        Raises:
            TypeError: An error due to the invalid input data type of arr.
        """
        # Check whether the input array is a list 
        if arr and not isinstance(arr, list):
            raise TypeError("The parameter 'arr' must be a python list")

        self.__heap = arr if arr else []
        self.__heapify()

    def __len__(self) -> int:
        """Return the number elements in the minheap.

        Returns:
            An integer representing the number of element in the current minheap.
            For example: 

            heap = MinHeap([1, 2, 3, 4, 5]) 
            print(len(heap))
            # Output: 5
        """
        return len(self.__heap)
    
    def push(self, val: Tuple[int, type[Node]]) -> None:
        """Push the value into MinHeap.

        Args:
            val: Tuple with the integer and a node that will be pushed to the heap.
        """
        #The current index where the new value will be put 
        curr = len(self.__heap)
        #Append the new value to the heap
        self.__heap.append(val)
        #Using the __get_parent_idx to get parent index
        parent = self.__get_parent_idx(curr)
        
        #Compare the new element with its parent to check if it is not bigger that its parent and swap if it is
        while self.__heap[curr] < self.__heap[parent]:
            self.__swap(curr, parent)
            curr = parent
            parent = self.__get_parent_idx(parent)
        
    def pop(self) -> Tuple[int, type[Node]]:
        """Returns and deletes the smallest element from the heap.

        Returns:
            A tuple consist of the an integer and a node object or None if the heap is empty.
            For example:

            heap = MinHeap([(1, Node_Obj1), (2, Node_Obj2), ...])
            print(heap.pop())
            # Output: (1, Node_Obj1)
        """
        if not self.__heap:
            return
        #Change the root noed for th last node using the __swap method
        self.__swap(0, len(self.__heap) - 1)
        #Creating the variable for the smalelst item
        min_element = self.__heap.pop()
        #Using __sift down method for keeping the heap property
        self.__sift_down(0)
        #return the element
        return min_element

    def top(self) -> Tuple[int, type[Node]]:
        """Return the smallest element from the heap.

        Returns:
            A tuple consist of the an integer and a node object or None if the heap is empty.
            For example:

            heap = MinHeap([(1, Node_Obj1), (2, Node_Obj2), ...])
            print(heap.top())
            # Output: (1, Node_Obj1)
        """
        if not self.__heap:
            return
        return self.__heap[0]

    def __heapify(self) -> None:
        """Heapify the input array by sifting down all the non-leaf nodes."""
        for idx in range(len(self.__heap) // 2, -1, -1):
            self.__sift_down(idx)

    def __sift_down(self, idx: int) -> None:
        """A heap operation thats maintain the min heap structure.

        It allocates the nodes with smallest values at the root and largest values at the leaves
        by sifting down every nodes in the wrong positions.
        """
        left_child_idx, right_child_idx = self.__get_left_child_idx(idx), self.__get_right_child_idx(idx)

        # Check whether the current node has 2 children
        if min(left_child_idx, right_child_idx) != -1:
            # Check whether the current node is smaller than both its children
            if min(self.__heap[idx],
                   self.__heap[left_child_idx],
                   self.__heap[right_child_idx]) != self.__heap[idx]:
                # If the left child is smaller than the right child,
                # swap the left child with the current node and sift down the left child
                if self.__heap[left_child_idx] < self.__heap[right_child_idx]:
                    self.__swap(idx, left_child_idx)
                    self.__sift_down(left_child_idx)
                # Otherwise, swap the right child with the current node and sift down the right child
                else:
                    self.__swap(idx, right_child_idx)
                    self.__sift_down(right_child_idx)
            
        # Else if the current node is larger than its only left child,
        # swap the left child with the current node and sift down the left child
        elif (left_child_idx != -1 and
              self.__heap[left_child_idx] < self.__heap[idx]):
            self.__swap(idx, left_child_idx)
            self.__sift_down(left_child_idx)
    
        # Else if the current node is larger than its only right child,
        # swap the right child with the current node and sift down the right child
        elif (right_child_idx != -1 and
              self.__heap[right_child_idx] < self.__heap[idx]):
            self.__swap(idx, right_child_idx)
            self.__sift_down(right_child_idx)
     
    def __swap(self, idx1: int, idx2: int) -> None:
        """Swap the elements with index idx1 and idx2 in the heap."""
        self.__heap[idx1], self.__heap[idx2] = self.__heap[idx2], self.__heap[idx1]

    def __get_parent_idx(self, idx: int) -> int:
        """Get the parent of the element with index idx.

        Returns:
            An integer representing the index of the parent, or -1 if no parent is found.
        """
        return idx // 2
    
    def __get_left_child_idx(self, idx: int) -> int:
        """Get the left child of the element with index idx.

        Returns:
            An integer representing the index of the left child, or -1 if no left child is found.
        """
        left_child_idx = 2 * idx + 1
        return left_child_idx if left_child_idx < len(self.__heap) else -1

    def __get_right_child_idx(self, idx: int) -> int:
        """Get the right child of the element with index idx.

        Returns:
            An integer representing the index of the right child, or -1 if no right child is found. 
        """
        right_child_idx = 2 * idx + 2
        return right_child_idx if right_child_idx < len(self.__heap) else -1

#### RandomisedSet Class
A set data structure which enables O(1) (constant time) insertion, removal and random extraction of elements at the expense of additional memory. <br>
It is used for storing the unconnected edges for the random graph generation in the Graph class. <br><br>
Major contributor: Tom

In [6]:
class RandomisedSet:
    """A randomised set which enables the insertion, deletion and random extraction of edges in O(1) (constant) time.

    Attributes:
        edges: A list storing non-duplicate edges that are not connected in the graph.
        edge_to_idx: A hashmap mapping the edges to the indices they are stored in the edges list.

    Methods:
        add_edges_from_node: Add available edges connected to all nodes from the input node.
        add_edge_to_set: Add an edge to the randomised set.
        remove_edge_from_set: Remove an edge from the randomised set.
        get_random_edge: Extract a random available edge.
    """

    def __init__(self) -> None:
        """Construct the attributes of the set."""
        self.edges = []
        self.edge_to_idx = {}

    def __contains__(self, item: Tuple[int, int]) -> bool:
        """Enable the use of membership test operators (in & not in) for the class.

        Args:
            item (tuple): A tuple to be checked whether it is contained in the edges set.

        Returns:
            True if the edge is found, false otherwise.
        """
        idx1, idx2 = item
        return (idx1, idx2) in self.edge_to_idx or (idx2, idx1) in self.edge_to_idx

    def add_edges_from_node(self, idx: int, nodes: List[int]) -> None:
        """Add outcoming edges to the set from a new node.
        
        For example:
        idx: 5
        nodes: [1, 3, 7, 8]
        # New edges created: (1, 5), (3, 5), (5, 7), (5, 8)

        Notes: 
            Adding n edges in O(n) time, where n = num of input nodes.
            Amortized time complexity for adding an edge: O(1).

        Args:
            idx (int): The index of the new node.
            nodes (list): The indices of the existing nodes.

        Raises:
            TypeError: Errors caused by incompatible input data type of 'idx' and 'nodes'.
        """
        # Check data types of parameters 'idx' and 'nodes'
        if not isinstance(idx, int):
            raise TypeError("The input parameter 'idx' must be an integer")
        if (not isinstance(nodes, list) or 
            not all([isinstance(node, int) for node in nodes])):
            raise TypeError("The input parameter 'nodes' must be a list of integers")

        # Add the new edges to the set
        for node_idx in nodes:
            # Skip to the next iteration if the index of starting node = ending node
            if node_idx == idx:
                continue
            # Format the new edge as (u, v), where u < v
            mn, mx = min(idx, node_idx), max(idx, node_idx)
            new_edge = (mn, mx)

            # Check whether the new edge exists
            if new_edge not in self.edge_to_idx:
                # If not, add it to the list and map the edge to its indices in the list
                self.edge_to_idx[new_edge] = len(self.edges)
                self.edges.append(new_edge)

    def add_edge_to_set(self, idx1: int, idx2: int) -> None:
        """Add an edge to the randomised set.

        Args:
            idx1 (int): Index of the first node.
            idx2 (int): Index of the second node.

        Raises:
            TypeError: Error occurs if the node indices are not integers.
            ValueError: Error occurs if the node indices are out of bounds.
        """
        # Check if indices are integers
        if not isinstance(idx1, int) or not isinstance(idx2, int):
            raise TypeError("Node indices must be integers.")

        # Check if indices are greater than 0
        if idx1 < 1 or idx2 < 1:
            raise ValueError("Node indices must be positive.")
        
        # making sure idx1 is less than idx2 to maintain consistency
        idx1, idx2 = min(idx1, idx2), max(idx1, idx2)
        new_edge = (idx1, idx2)

        # Add the new edge to the randomised set 
        if new_edge not in self.edge_to_idx:
            self.edge_to_idx[new_edge] = len(self.edges)
            self.edges.append(new_edge)

    def remove_edge_from_set(self, idx1: int, idx2: int) -> None:
        """Remove an edge from the set.

        Notes:
            First swap the target edge with the last edge in the edges list and pop it out of the list.
            
        Args:
            idx1 (int): Index of the first node.
            idx2 (int): Index of the second node.

        Raises:
            TypeError: Error occurs if the node indices are not integers.
            ValueError: Error occurs if the edge does not exist.
        """
        # Check if indices are integers
        if not isinstance(idx1, int) or not isinstance(idx2, int):
            raise TypeError("Node indices must be integers.")

        # Check if indices exist in the set
        removing_edge = (min(idx1, idx2), max(idx1, idx2))
        if removing_edge not in self.edge_to_idx:
            raise ValueError("The edge does not exist in the set") 
        
        # Get the indices of the edge to be removed and the last edge in the list
        removing_edge_index = self.edge_to_idx[removing_edge]
        last_edge = self.edges[-1]

        # Replace the edge to be removed with the last edges and change the corresponding index in the node map
        self.edge_to_idx[last_edge] = removing_edge_index
        self.edges[removing_edge_index] = last_edge

        # Remove the last entry in the edges list and delete the index map for the removed edge 
        self.edges.pop()
        del self.edge_to_idx[removing_edge]

    def get_random_edge(self) -> Tuple[int, int]:
        """Extract and remove a random available edge from the set.

        Returns:
            A tuple consists of the start and end of the edge. For example: (1, 2).
        """
        if not self.edges:
            return
        
        # Randomly select an edge from the list
        edge_idx = rand.randint(0, len(self.edges) - 1)
        edge = self.edges[edge_idx]

        # Delete the edge from the set and return it
        self.remove_edge_from_set(*edge)
        return edge

#### Trie Class
A prefix tree data structure which stores alphabets as node values. <br>
It is utilized in the SearchEngine class for searching player in the leaderboard.<br><br>
Major contributor: Tom

In [7]:
class TrieNode:
    """An individual trie node.
    
    Attributes:
        children: A dictionary mapping the char to its TrieNode object.
        end_of_word: A boolean value indicating whether the current char is the end of the word.
    """
    def __init__(self):
        """Consturct the children and end_of_word attributes."""
        self.children = {}
        self.end_of_word = False


class Trie:
    """A prefix tree data structure which stores an alphabet as value in each node.
    
    Attributes:
        root: A pointer to the root of the Trie.

    Methods:
        find: Return True if the word is in the dictionary else False
        complete: Complete a word based on the input of the user and return the list of words ordered by their length
    """
    def __init__(self) -> None:
        """Construct the root of the trie."""
        self.root = TrieNode()

    def insert(self, word: str) -> None:
        """Insert the word into the trie.

        Args:
            word (str): A word to be inserted into the trie.

        Raises:
            TypeError: Errors caused by non-string input of 'word'.
        """
        if not isinstance(word, str):
            raise TypeError("The input parameter 'word' must be a string")

        # Create a pointer to the root
        node = self.root

        # Check if the character is a child of the root
        for char in word:
            # Create a new branch if the character is not found
            if char not in node.children:
                node.children[char] = TrieNode()
            # Traverse to the node storing the character 
            node = node.children[char]
            
        # Set the node storing the last character of the word to be the end_of_word
        node.end_of_word = True

    def find(self, word: str) -> bool:
        """Search whether the word is stored in the trie.

        Args:
            word (str): A word to be searched for in the trie.
        
        Returns:
            True if the word is found else false.

        Raises:
            TypeError: Errors caused by non-string input of 'word'.
        """
        if not isinstance(word, str):
            raise TypeError("The input parameter 'word' must be a string")

        node = self.root
        # Check every character in the word
        for char in word:
            # Return false if any char is not found
            if char not in node.children:
                return False
            # Traverse to the next char
            node = node.children[char]
        
        # Return true if the last char is marked as the end of word
        return node.end_of_word
    
    def complete(self, word: str) -> List[str]:
        """Complete the given word by searching words in the trie with the same prefix.

        Args:
            word (str): A word to be completed.
        
        Returns:
            A list of possible words in the trie.

        Raises:
            TypeError: Errors caused by non-string input of 'word'.
        """
        if not isinstance(word, str):
            raise TypeError("The input parameter 'word' must be a string")

        node = self.root
        res = []

        # Find the node storing the last character of the input string
        for char in word:
            # Return an empty list if the input string is not found
            if char not in node.children:
                return []
            node = node.children[char]

        # Use backtracking to find all the combinations of words starting at the last char of the input str
        cache = []
        def dfs(node: type[TrieNode]) -> None:
            # Append the word to res if it is marked as the end of word
            if node.end_of_word and cache:
                res.append(word + ''.join(cache)) 

            for child in node.children:
                # Backtracking 
                # Use a cache variable to store the combinations of the word
                cache.append(child)
                # Continue recursion on each child of the subtree
                dfs(node.children[child])
                # Remove the char from the cache once a combination have been added to res
                cache.pop()

        dfs(node)
        # Return the list sorted by the length of each word
        return sorted(res, key=len)
    
    def fizzy_search(self, 
                     word: str,
                     threshold: int, 
                     num_return: int = int(1e9)) -> List[str]:
        """A search method which returns all similar words in the trie with at most (threshold) Levenshtein distance. 

        Args:
            word (str): The target word.
            threshold (int): The maximum Levenshtein distance difference.
            num_return (int): The maximum number of return words sorted by ascending levenshtein distance.

        Returns:
            A list of words that are within the Levenshtein distance threshold.

        Raises:
            TypeError: Invalid data type of input parameter 'word'.
            ValueError: Invalid data type and range of input parameters 'threshold' or 'num_return'.
        """
        if not isinstance(word, str):
            raise TypeError("The input parameter 'word' must be a string")
        
        if not isinstance(threshold, int) or threshold < 0:
            raise ValueError("The input parameter 'threshold' must be a non-negative integer")
        if not isinstance(num_return, int) or num_return < 1:
            raise ValueError("The input parameter 'num_return' must be a positive integer")
        '''
        Method: using a 2d matrix to find the the Levenshtein distance between 2 strings
        e.g.

            k a t e
          0 1 2 3 4
        c 1 1 2 3 4
        a 2 2 1 2 3
        t 3 3 2 1 2

        x: target_word
        y: possible combination in trie

        Steps:
        1. First fill the first row with range(len(target_word))
        2. Iterate through the rows from left to right
        3. For each grid, if the letter of the target word != the letter of the combination,
            compute the lowest cost by taking the lowest values at the top left, left and the grid above + 1.
            Otherwise, just take the value of the top left grid (the previous cost) without adding 1.
        4. The difference between the 2 words will be grid[len(target_word)][len(combination)]
    
        Results:
        Difference between 'kate' and 'cat' = 2 (grid[4, 3])
        - Replace c by k and insert e

        Difference between 'kat' and 'ca' = 2 (grid[3, 2])
        - Replace c by k and insert t

        As this algorithm performs dfs on the trie while completing the grid, 
        new character will be added to the y-axis in every iteration. 
        We just have to keep track of the previous row and the current row of characters.
        '''
        # Use a minheap to extract words with the lower difference
        heap = MinHeap()
        root = self.root
        res = []
        # Create the first row of the 2d matrix
        curr_row = range(len(word) + 1)

        def dfs(node: type[TrieNode],
                letter: str,
                curr_str: str,
                prev_row: int) -> None:
            # Add the new letter to the string storing the combination
            curr_str += letter

            cols = len(word) + 1
            # Create the first value of the new row
            curr_row = [prev_row[0] + 1]

            for col in range(1, cols):
                # Add the value of the top left grid to curr_row 
                # if the combination and target share the same letter 
                if word[col - 1].lower() == letter.lower():
                    curr_row.append(prev_row[col - 1])
                    continue
                
                # Otherwise calculate each cost of the 3 grids at the 3 directions
                # according to the edit distance algorithm
                replace_cost = prev_row[col - 1] + 1
                insert_cost = curr_row[col - 1] + 1
                delete_cost = prev_row[col] + 1

                # Append the minimum cost to the curr_row
                curr_row.append(min(replace_cost, insert_cost, delete_cost))

            # If the last value of the row is less than the threshold
            # and it is a word stored in the trie, push the word along with the Levenshtein distance to the heap 
            if curr_row[-1] <= threshold and node.end_of_word:
                heap.push((curr_row[-1], curr_str))

            # If the minimum value of the row has not exceeded the threshold, 
            # it is possible that adding additional characters to the end of the word will still be valid
            if min(curr_row) <= threshold:
                for child in node.children:
                    dfs(node.children[child], child, curr_str, curr_row)

        # Start the recursion from every child of the trie root
        for child in root.children:
            dfs(root.children[child], child, '', curr_row)

        # Pop the words with the lowest Levenshtein distance from the heap and return
        while len(heap) and num_return:
            res.append(heap.pop()[1])
            num_return -= 1

        return res

### __Additional Features Modules__
These modules contain the database for storing player data and the leaderboard search engine for searching players.

In [8]:
import sqlite3
import os

#### DatabaseConnection Class
A context manager class which automatically handles the connection and disconnection of the database.
<br><br>
Major contributor: Femi

In [9]:
class DatabaseConnection:
    """A context manager class for handling database connections."""
    
    def __init__(self, db_name):
        """Initialize the DatabaseConnection object."""
        self.db_name = db_name
        self.connection = None

    def __enter__(self):
        """Enter method for the context manager."""
        try:
            self.connection = sqlite3.connect(self.db_name)
            return self.connection
        except sqlite3.Error as e:
            print(f"Error connecting to database: {e}")
            raise
       
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Exit method for the context manager."""
        if self.connection:
            try:
                self.connection.commit()
            except sqlite3.Error as e:
                print(f"Error committing changes to database: {e}")
            finally:
                self.connection.close()

#### Data Access Modules
Functions that provides authentication and access to player balance and history. <br><br>
Major contributor: Femi

In [10]:
def initialize_database():
    """Initialize the database with necessary tables."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("CREATE TABLE IF NOT EXISTS players (id INTEGER PRIMARY KEY, balance INTEGER, username TEXT UNIQUE, password TEXT)")
            cursor.execute("CREATE TABLE IF NOT EXISTS games (id INTEGER PRIMARY KEY, username TEXT, bid INTEGER, start INTEGER, end INTEGER, outcome TEXT, score INTEGER, entry_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP)")
        print("Database initialized successfully.")
    except sqlite3.Error as e:
        print(f"Error initializing database: {e}")

def registered(username):
    """Check whether the player has registered an account."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("SELECT * FROM players WHERE username = ?", (username,))
            player = cursor.fetchone()
        return bool(player)
    except sqlite3.Error as e:
        print(f"Error validating player's registration status: {e}")

def register_player(username, password, initial_balance):
    """Register a new player in the database."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("INSERT INTO players (balance, username, password) VALUES (?, ?, ?)", (initial_balance, username, password))
        print("Player registered successfully.")
    except sqlite3.Error as e:
        print(f"Error registering player: {e}")
    

def log_game(username, bid, start, end, outcome, score):
    """Log a game played by a player in the database."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("INSERT INTO games (username, bid, start, end, outcome, score) VALUES (?, ?, ?, ?, ?, ?)", (username, bid, start, end, outcome, score))
        print("Game logged successfully.")
    except sqlite3.Error as e:
        print(f"Error logging game: {e}")


def get_player_history(username):
    """Extract the game history of a player."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("SELECT bid, start, end, outcome, score, entry_date FROM games WHERE username = ? ORDER BY entry_date DESC", (username,))
            records = cursor.fetchall()
        print("Fetch records successfully.")
        return records
    except sqlite3.Error as e:
        print(f"Error fetching records: {e}")


def authenticate(username, password):
    """Authenticate the user based on provided username and password."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("SELECT username, balance FROM players WHERE username = ? AND password = ?", (username, password))
            user = cursor.fetchone()
        if user:
            print("Authentication successful.")
            return user
        else:
            print("Invalid username or password.")
            return False
    except sqlite3.Error as e:
        print(f"Error authenticating user: {e}")
        return False
    
    
def update_balance(username, new_balance):
    """Update the balance of a player."""
    try:
        with DatabaseConnection('db') as connection:
            cursor = connection.cursor()
            cursor.execute("UPDATE players SET balance = ? WHERE username = ?", (new_balance, username))
        print("Balance updated successfully.")
    except sqlite3.Error as e:
        print(f"Error updating balance: {e}")

#### SearchEngine Class
A search engine that autocomplete the user input and provides search results for the leaderboard. <br>
It is utilized in the GraphGameGUI class.<br><br>
Major contributor: Tom

In [11]:
class SearchEngine:
    def __init__(self) -> None:
        """A search engine that provides searching functionality for the leaderboard in the Graph Game.

        Attributes:
            trie: An instance of the trie data structure.
        
        Methods:
            complete_search: Complete a word based on the input and return the list of word combinations ordered by their length.
            search_results: Get a list of the players with their balance according to the user input.
            get_leaders: Get the n players with the highest balance.
            fetch_all_users_to_trie: Fetch all the players from the database to the trie.
        """
        self.trie = None
        self.fetch_all_users_to_trie()
        
    def complete_search(self, input_str: str) -> List[str]:
        """Complete a word based on the input and return the list of word combinations ordered by their length.

        Args:
            input_str (str): The user input.

        Returns:
            A list of word combinations ordered by their length.
        """
        return self.trie.complete(input_str)
    
    def search_results(self, name: str) -> List[Tuple[str, int]]:
        """Get a list of the players with their balance according to the user input.

        Args:
            name (str): The target player name.

        Returns:
            A list of tuples consist of the players' name and balance. 
        """
        # Get at most 10 users with at most 2 Levenshtein distance away from the name input
        players = self.trie.fizzy_search(name, threshold=5, num_return=10)

        res = []

        with DatabaseConnection('db') as conn:
            for player in players:
                # Fetch the player name and score from the database
                player_name_score = conn.cursor().execute(
                    'SELECT username, balance FROM players WHERE username = ?', (player,)
                ).fetchone()

                # Append the records of the players to the res list
                res.append(player_name_score)

        return res

    def get_leaders(self, n: int) -> List[Tuple[str, int]]:
        """Get the n players with the highest balance.

        Args:
            n (int): The number of top players.

        Returns:
            A list of tuples consist of the players' name and balance. 

        Raises:
            ValueError: Invalid data type or range of n.
        """
        if not isinstance(n, int) or n < 1:
            raise ValueError("Input parameter 'n' must be a postive integer")

        with DatabaseConnection('db') as conn:
            leaders_name_score = conn.cursor().execute(
                'SELECT username, balance FROM players ORDER BY balance DESC'
            ).fetchall()
            
        return leaders_name_score[:n]

    def fetch_all_users_to_trie(self) -> None:  
        """Fetch all the players from the database to the trie."""
        with DatabaseConnection('db') as conn:
            # Get a list of all players
            players = conn.cursor().execute(
                'SELECT username FROM players'
            ).fetchall()
        
        self.trie = Trie()

        # Insert every player into the trie
        for player in players:
            self.trie.insert(*player)

### __Game Logic Modules__
These modules contain the game logic and the user interface of the game.

In [12]:
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure 
import networkx as nx
import os
import pygame
from scipy import stats
import tkinter as tk
from tkinter import messagebox, ttk

pygame 2.5.2 (SDL 2.28.3, Python 3.11.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### RandomScoreGenerator Class
A score generator that generates random scores for each graph node based on a normal distribution. <br>
It is utilized in the GraphGame class for generating node score and a random cutoff distance for each round of the game.
<br><br>
Major contributors: Femi, Tom

In [13]:
class RandomScoreGenerator:
    """A random score generator which based on a normal distribution.

    Attributes:
        base_score: The base score of the game which determines the range of score generated for each node.

    Methods:
        set_mean: Set the mean value for generating random distances.
        set_sd: Set the standard deviation for generating random distances.
        generate_random_distance: Generates a random distance using a normal distribution.
        calculate_score: Calculates a score based on the given distance and base score.
        generate_random_edge: Generates a random weight using a normal distribution.
    """
    def __init__(self, base_score: int = 100) -> None:
        """Construct the attributes of the random score generator.

        Args:
            base_score (int): The base score of the game which determines the range of score generated for each node.

        Raises:
            ValueError: If the base score is not a integer.
        """
        self._mean = None
        self._sd = None
        self._generated_distance = None
        self.base_score = base_score

    def set_mean(self, mean: int) -> None:
        """Set the mean value for generating random distances.

        Args:
            mean (int): The mean value to set.

        Raises:
            ValueError: If the mean is a non numeric number.
        """
        # Check if mean is a non-negative integer
        if not isinstance(mean, int) or mean < 0:
            raise ValueError("Mean must be a non-negative integer.")
        # Set mean
        self._mean = mean

    def set_sd(self, sd: int | float) -> None:
        """Set the standard deviation for generating random distances.

        Args:
            sd (int | float): The standard deviation value to set.

        Raises:
            ValueError: If the standard deviation is not a non negative number.
        """
        # Check if sd is a non-negative numeric value
        if not isinstance(sd, (int, float)) or sd < 0:
            raise ValueError("Standard deviation must be a non-negative numeric value.")
        # Set standard deviation
        self._sd = sd

    def generate_random_distance(self) -> int:
        """Generates a random distance using a normal distribution.

        Returns:
            int: A randomly generated distance.
        
        Raises:
            ValueError: If mean or standard deviation is not set.
        """
        # Check if mean and standard deviation are set
        if self._mean is None or self._sd is None:
            raise ValueError("Mean and standard deviation must be set.")
        # Generate random distance
        return max(int(np.random.normal(loc=self._mean, scale=self._sd)), 0)

    def calculate_score(self, dist: int) -> int:
        """Calculates a score based on the given distance and base score.

        Args:
            dist (int): The distance to calculate the score for.

        Returns:
            int: The calculated score.

        Raises:
            ValueError: If the distance is not an integer.
        """
        # Check if distance is a non-negative integer
        if not isinstance(dist, int) or dist < 0:
            raise ValueError("Distance must be a non-negative integer.")
            
        # If generated distance is not already set, generate it    
        if self._generated_distance is None:
            self._generated_distance = self.generate_random_distance()
            
        # Calculate score
        norm_distribution = stats.norm(self._mean, self._sd)
        # Calculate the probability that the node dist > a random generated distance
        prob = 1 - norm_distribution.cdf(dist)
        # Take the inverse of prob so the longer the node distance (the lower the prob), the higher the score
        return int(1 / prob * self._sd * 2 + self.base_score)
    
    @staticmethod
    def generate_random_edge(mean: int, sd: int | float) -> int:
        """Generates a random weight using a normal distribution.

        Returns:
            int: A randomly generated weight.
        
        Raises:
            ValueError: If mean is not an integer or standard deviation is neither an integer nor a float num.
        """
        # Check if mean is a non-negative integer
        if not isinstance(mean, int) or mean < 0:
            raise ValueError("Mean must be a non-negative integer.")
        # Check if sd is a non-negative numeric value
        if not isinstance(sd, (int, float)) or sd < 0:
            raise ValueError("Standard deviation must be a non-negative numeric value.")
        # Generate random weight, ensuring it's at least 1
        return max(int(np.random.normal(loc=mean, scale=sd)), 1)

#### Graph Class
The class initializes a random graph, compromises of fully-connected nodes and weighted edges. <br>
It is inherited by the GraphGame class to create the game logic. <br><br>
Major contributor: Tom

In [14]:
class Graph:
    """A randomly generated undirected and fully connected graph data structure.

    Notes:
        Class attributes should not be directed accessed.

    Attributes:
        G: An instance of the Graph class of the networkx module.
        node_position: The positions of each nodes in the networkx graph.
        unconnected_edges: An instance of the RandomisedSet class storing all unconnected edges.
        node_map: A hashmap mapping the index of the node to its Node object.
        node_idx_count: An integer count for indexing new nodes.
        num_nodes: The total number of nodes in the graph.
        num_edges: The total number of edges in the graph.
        edge_mean: The mean weight of the generated edges.
        edge_sd: The standard deviation of the weight of the generated edges.

    Methods:
        generate_random_nodes: Generate a random number of nodes.
        generate_random_edges: Generate a random number of weighted edges.
        add_edge_to_graph: Add an edge with random weight to the graph.
        shortest_path: Find the shortest path between 2 nodes.
        graph_visualize: Graphical visualization of the graph using matplotlib libary for testing.
    """

    def __init__(self,
                 init_num_nodes: int = 0,
                 add_num_edges: int = 0,
                 edge_mean: int = 5,
                 edge_sd: int | float = 3) -> None:
        """Construct all attributes of the graph data structure.

        Args:
            init_num_nodes (int): Number of randomly generated nodes during initialization.
            add_num_edges (int): Number of additional randomly generated edges, after 
                           generating n - 1 edges to connect all nodes, n = init_num_nodes.
            edge_mean (int): The mean weight of the generated edges.
            edge_sd (int, float): The standard deviation of the weight of the generated edges.

        Raises:
            TypeError: Errors caused by incompatible data types of input parameters.
            ValueError: Errors caused by negative inputs of parameters.
        """
        if not all([isinstance(init_num_nodes, int),
                    isinstance(add_num_edges, int),
                    isinstance(edge_mean, int)]):
            raise TypeError("Input parameters 'init_num_nodes', 'init_num_edges' and 'edge_mean' must be integers")
        
        if not isinstance(edge_sd, (int, float)):
            raise TypeError("Input parameter 'edge_sd' must be an integer or a float")

        if min(init_num_nodes, 
               add_num_edges, 
               edge_mean, 
               edge_sd) < 0:
            raise ValueError("All input parameters must be non-negative")
        
        self.G = nx.Graph()
        self.node_position = None
        self.unconnected_edges = RandomisedSet()
        self.node_map = {}
        self.node_idx_count = 1
        self.num_nodes = 0
        self.num_edges = 0
        self.edge_mean = edge_mean
        self.edge_sd = edge_sd

        self.generate_random_nodes(init_num_nodes)
        self.__randomly_connect_all_nodes()
        self.generate_random_edges(add_num_edges)
    
    def __str__(self) -> str:
        """Return the basic information about the Graph object.
        
        Returns:
            A string including the number of edges (|E|), the number of vertices (|V|),
            the mean weight of the edges (E.x̄) and the standard deviation of the edge weights (E.σ).
        """
        return f'Graph: |E| = {self.num_edges}, |V| = {self.num_nodes}, E.x̄ = {self.edge_mean}, E.σ = {self.edge_sd}'

    def generate_random_nodes(self, 
                              num: int | None = None,
                              low: int = 10,
                              high: int = 15) -> None:
        """Generate a random number of nodes in the graph.

        Args:
            num (int): An integer representing the fixed number of nodes to be generated (optional).
            low (int): The minimum number of nodes generated by the method (default = 10).
            high (int): The minimum number of nodes generated by the method (default = 15).

        Raises:
            TypeError: Errors caused by non-integer parameters input.
            ValueError: Errors caused by non-negative parameters input or the invalid ranges of 'low' and 'high'.
        """
        # Check data type of input parameters
        if not all([isinstance(num, (int, None)),
                   isinstance(low, int),
                   isinstance(high, int)]):
            raise TypeError("All input parameters must be integers")
    
        # Check values of input parameters
        if min(low, high) < 0:
            raise ValueError("Input parameters 'low' and 'high' must be non-negative")
        if low > high:
            raise ValueError("Input parameter 'high' must be equal or larger than 'low'")
        
        # Generate random num of nodes if the user does not define a fixed num
        if num is None:
            num = rand.randint(low, high)

        # Raise a value error if the user enters a negative number of nodes
        if num < 0:
            raise ValueError("Input parameter 'num' must be non-negative")

        self.num_nodes += num

        # Generate nodes
        while num:
            # Add the new node to the node map and the networkx graph
            self.node_map[self.node_idx_count] = Node(self.node_idx_count)
            self.G.add_node(self.node_idx_count)

            # Add all possible edges to the unconnected edges set
            node_indices = list(self.node_map.keys())
            self.unconnected_edges.add_edges_from_node(self.node_idx_count, node_indices)

            self.node_idx_count += 1
            num -= 1

    def generate_random_edges(self,
                              num: int | None = None,
                              low: int = 5,
                              high: int = 10) -> None:
        """Generate a random number of edges in the graph.

        Args:
            num (int): An integer representing the fixed number of edges to be generated (optional).
            low (int): The minimum number of edges generated by the method (default = 5).
            high (int): The minimum number of edges generated by the method (default = 10).

        Raises:
            TypeError: Errors caused by non-integer parameters input.
            ValueError: Errors caused by non-negative parameters input or the invalid ranges of 'low' and 'high'.
        """
        # Check data type of input parameters
        if not all([isinstance(num, (int, None)),
                   isinstance(low, int),
                   isinstance(high, int)]):
            raise TypeError("All input parameters must be integers")


        # Check values of input parameters
        if min(low, high) < 0:
            raise ValueError("All input parameters must be non-negative")
        if low > high:
            raise ValueError("Input parameter 'high' must be equal or larger than 'low'")
        
        # Generate random num of edges if the user does not define a fixed num
        if num is None:
            num = rand.randint(low, high)

        # Raise a value error if the user enters a negative number of nodes
        if num < 0:
            raise ValueError("Input parameter 'num' must be non-negative")

        # Get the maximum number of edges that could exist in the graph
        max_num_edges = self.__get_max_num_edges()
            
        # Generate edges while the number of edges in the graph has not reached the maximum
        while num and self.num_edges < max_num_edges:
            # Get the start and end node of the random edge
            start, end = self.unconnected_edges.get_random_edge()

            # Add an edges between the 2 nodes
            self.add_edge_to_graph(start, end)

            # Decrease the remaining num of edges to be generated
            num -= 1

        # Assigning a position for each node of the graph using the srping layout algorithm
        self.node_position = nx.spring_layout(self.G)

    def add_edge_to_graph(self, idx1: int, idx2: int) -> None:
        """Add an edge to the graph with random integer weight from 1 to 10.

        Args:
            idx1 (int): The index of the first node.
            idx2 (int): The index of the second node.

        Raises:
            TypeError: Errors caused by non-integer parameters input.
            ValueError: Errors caused by invalid range or non-existing node parameters input.
        """
        # Check data type of input parameters
        if (not isinstance(idx1, int) or 
            not isinstance(idx2, int)):
            raise TypeError("Input parameters 'idx1' and 'idx2' must be integers")
        
        # Check value ranges of input parameters
        if idx1 == idx2:
            raise ValueError("The value of 'idx1' cannot be the same as 'idx2'")
        if (idx1 not in self.node_map or 
            idx2 not in self.node_map):
            raise ValueError("The input node(s) do(es) not exist in the graph")
        
        # Get the start and end node objects from the node_map 
        start_node = self.node_map[idx1]
        end_node = self.node_map[idx2]

        # Randomly generate weight for the new edge
        weight = RandomScoreGenerator.generate_random_edge(mean=self.edge_mean, sd=self.edge_sd)

        # Set the nodes as their neighors
        start_node.add_neighbour(end_node, weight)
        end_node.add_neighbour(start_node, weight)

        # Add the edge to the networkx visualization
        self.G.add_edge(idx1, idx2, weight=weight)

        # Remove the edge from the unconnected edges set
        if (idx1, idx2) in self.unconnected_edges:
            self.unconnected_edges.remove_edge_from_set(idx1, idx2)

        # Increase the num of edges in the graph
        self.num_edges += 1

    def shortest_path(self, starting_node: int, ending_node: int | None = None) -> int | Dict[int, int]:
        """Find the shortest path between nodes in the graph.

        Notes:
            If the user does not pass in the ending_node parameter, the program will return 
            a dictionary mapping the shortest distance to all nodes. 

        Args:
            starting_node (int): Index of the starting node.
            ending_node (int): Index of the ending node (optional).

        Returns:
            The length of the shortest path or a dictionary containing the shortest paths to all nodes.
        """
        # Check if both starting node and ending node are in the graph
        if (starting_node not in self.node_map or
            ending_node is not None and ending_node not in self.node_map):
            raise ValueError("The input nodes does not exist in the graph")

        # Create priority queue as the object of MinHeap class
        priority_queue = MinHeap([(0, self.node_map[starting_node])])  
        visited_nodes = set()

        # Create a dictionaray storing the the shortest path from the source node to every other nodes
        paths = {node_idx: float('inf') for node_idx in self.node_map.keys()}
        paths[starting_node] = 0

         # Checking each node until all nodes are visited
        while len(priority_queue) > 0:  
            # Pop the node with the smallest distance to the ending_node
            current_distance, current_node_object = priority_queue.pop()  
            current_node = current_node_object.get_index()

            # If the current node is the ending_node, the current distance will be returned
            if current_node == ending_node:
                return current_distance
            
            # Update the shortest path of a node in paths if a shorter path is found
            if current_distance < paths[current_node]:
                paths[current_node] = current_distance

             # Go through each neighbor of the current_node
            if current_node not in visited_nodes:
                visited_nodes.add(current_node)

                # If the neighbor was not visited, it will be added to priority queue
                for neighbor_object, weight in current_node_object.get_neighbours():
                    if neighbor_object.get_index() not in visited_nodes:
                        new_distance = current_distance + weight

                        # Use push method to add neighbors
                        priority_queue.push((new_distance, neighbor_object))  
        
        # Return the path dict if the user does not specify an ending node
        del paths[starting_node]
        return paths

    def __get_max_num_edges(self) -> int:
        """Calculate the maximum number of edges can be connected in the graph.

        Returns:
            An integer representing the maximum number of edges.
        """
        n = self.num_nodes
        return n * (n - 1) // 2
    
    def __randomly_connect_all_nodes(self) -> None:
        """Randomly shuffle the list of nodes and connect them."""
        # Get the all the node indicies in the graph and shuffle them
        node_indices = list(self.node_map.keys())
        shuffled_nodes = rand.shuffle(node_indices)

        # Connect each node with its previous node
        for i in range(1, len(node_indices)):
            # Get the node objects from the node map
            node1 = self.node_map[node_indices[i]]
            node2 = self.node_map[node_indices[i - 1]]

            # Generate the edge weight based on a normal distribution
            weight = RandomScoreGenerator.generate_random_edge(mean=self.edge_mean, sd=self.edge_sd)

            # Add the nodes to their neighbour dictionaries
            node1.add_neighbour(node2, weight=weight)
            node2.add_neighbour(node1, weight=weight)

            # Add an edge in the networkx graph
            self.G.add_edge(node_indices[i], node_indices[i - 1], weight=weight)

            # Remove the edge from the unconnected edges set
            self.unconnected_edges.remove_edge_from_set(node_indices[i], node_indices[i - 1])
            
            self.num_edges += 1
            
    def graph_visualize(self, with_labels=True, node_size=700) -> None:
        """Graphical visualization of the graph using matplotlib libary for testing.

        Args:
            with_labels (bool): If True, nodes will show their ID as label.
            node_size (int): The size of the nodes.
        """
        # Drawing nodes of the graph
        nx.draw_networkx_nodes(self.G, self.node_position, node_size=node_size)
        nx.draw_networkx_labels(self.G, self.node_position, font_size=12)

        # Drawing edges of the nodes and set the width of each edge to be proportional to its weight
        edge_width = list(nx.get_edge_attributes(self.G, 'weight').values())
        nx.draw_networkx_edges(self.G, self.node_position, alpha=0.5, width=edge_width)
        
        # If the labels exist - draw them
        if with_labels:
            edge_labels = nx.get_edge_attributes(self.G, 'weight')
            nx.draw_networkx_edge_labels(self.G, self.node_position, edge_labels)

        # Axis are not needed for the project
        plt.axis('off') 

        # Visualize the graph
        plt.show() 

#### GraphGame Class
A subclass of the Graph class that handles the game operations and the winning conditions. <br><br>
Major contributor: Tom

In [15]:
class GraphGame(Graph):
    """A subclass inherited from the Graph class which handles the operation and the winning conditions of the game.

    Attributes:
        score_generator: An instance of the RandomScoreGenerator class used for generating scores for each node.
        starting_node: An integer index of the starting node defined by the user.
        ending_node: An integer index of the ending node defined by the user.
        cutoff_distance: The random distance generated by RandomScoreGenerator to determine the game outcome.
        base_score: An integer base score of the game.

    Methods:
        get_nodes: A getter method for all node indcies in the graph.
        set_base_score: A setter method for the base_score attribute.
        set_starting_node: A setter method for the starting node.
        set_ending_node: A setter method for the ending node.
        generate_cutoff: Generate and store the random distance in the cutoff_distance variable.
        check_player_wins: Check whether the player wins.
        get_player_score: Get the score awarded for the player for the current round.
        random_start: A classmethod for generating a random game.
    """

    def __init__(self,
                 init_num_nodes: int = 0, 
                 init_num_edges: int = 0) -> None:
        """Construct the attributes of the graph.

        Args:
            init_num_nodes (int): Number of randomly generated nodes during initialization.
            add_num_edges (int): Number of additional randomly generated edges, after 
                           generating n - 1 edges to connect all nodes, n = init_num_nodes.
        """
        super().__init__(init_num_nodes, init_num_edges)
        self.score_generator = None
        self.starting_node = None
        self.ending_node = None
        self.cutoff_distance = None
        self.base_score = None

    def get_nodes(self) -> List[int]:
        """The method for getting the list of nodes in the graph which is needed for tkinter combobox.

        Returns: 
            The list of nodes in the graph.
        """
        return sorted(list(self.node_map.keys()))

    def set_base_score(self, score: int) -> None:
        """A setter method for the base_score attribute.

        Args:
            score (int): A integer to be set as the base score.
        
        Raises:
            TypeError: Error caused by non-integer parameter input.
        """
        if not isinstance(score, int):
            raise TypeError("Input parameters 'score' must be an integer")
        
        self.base_score = score
        self.score_generator = RandomScoreGenerator(base_score=score)

    def set_starting_node(self, idx: int) -> None:
        """A setter method for the starting_node attribute.

        Args:
            idx: A integer index of the node to be set as the starting node.
        
        Raises:
            TypeError: Error caused by non-integer parameter input.
            ValueError: Error caused by non-existing node parameter input.
        """
        if not isinstance(idx, int):
            raise TypeError("Input parameters 'idx' must be an integer")
        if idx not in self.node_map:
            raise ValueError("The node with index 'idx' does not exist in the graph") 

        self.starting_node = idx

    def set_ending_node(self, idx: int) -> None:
        """A setter method for the ending_node attribute.

        Args:
            idx (int): A integer index of the node to be set as the ending node.
        
        Raises:
            TypeError: Error caused by non-integer parameter input.
            ValueError: Error caused by non-existing node parameter input.
        """
        if not isinstance(idx, int):
            raise TypeError("Input parameters 'idx' must be an integer")
        if idx not in self.node_map:
            raise ValueError("The node with index 'idx' does not exist in the graph") 
        
        self.ending_node = idx

    def generate_cutoff(self) -> None: 
        """ Generate and store the random distance in the cutoff_distance variable.
        
        Raises:
            NameError: if the score_generator has not been defined.
        """
        if not self.score_generator:
            raise NameError('The score_generator has not been defined')

        # Get the shortest distances from the starting node to all other nodes
        node_to_dist = self.shortest_path(starting_node=self.starting_node)
        dist_values = list(node_to_dist.values())

        # Calculate the mean and sd of the shortest distances using the numpy library
        mean_dist = int(np.mean(dist_values))
        sd_dist = np.std(dist_values)

        # Configure the mean and sd of the score_generator
        self.score_generator.set_mean(mean_dist)
        self.score_generator.set_sd(sd_dist*1.7)

        # Generate a random distance
        self.cutoff_distance = self.score_generator.generate_random_distance()
        
    def check_player_wins(self) -> bool:
        """Check whether the player wins.
        
        Returns:
            True if the player wins, false otherwise.

        Raises:
            NameError: Error raised if the starting_node, ending_node or cutoff_distance have not been defined.
        """
        if (not self.starting_node or not self.ending_node):
            raise NameError('The starting node and ending node have not been defined')
        if self.cutoff_distance is None:
            raise NameError('The generate_cutoff method has to be called before this method')
        
        # Find the shortest distance between the starting node and the ending node
        shortest_dist = self.shortest_path(self.starting_node, self.ending_node)

        # Return a boolean of whether the player wins
        return shortest_dist < self.cutoff_distance
    
    def get_player_score(self) -> int:
        """Get the score awarded for the player for the current round.

        Returns:
            An integer representing the score.

        Raises:
            NameError: Error raised if the starting_node, ending_node or cutoff_distance have not been defined.
        """
        if (not self.starting_node or not self.ending_node):
            raise NameError('The starting node and ending node have not been defined')
        if self.cutoff_distance is None:
            raise NameError('The generate_cutoff method has to be called before this method')

        # Find the shortest distance between the starting node and the ending node
        shortest_dist = self.shortest_path(self.starting_node, self.ending_node)

        # Generate the potential score of the player based on the distance
        score = self.score_generator.calculate_score(shortest_dist)

        # Return the calculated score if the player wins, otherwise return negative base score which indicates a loss
        return score if self.check_player_wins() else -self.base_score

    @classmethod
    def random_start(cls) -> type[GraphGame]:
        """An alternative initization method which generates a new game with a random graph.

        Returns:
            A GraphGame object with random nodes and weighted edges.

        To instantiate:
            game = GraphGame.random_start()
        """
        init_num_nodes = rand.randint(8, 10)
        add_num_edges = init_num_nodes // rand.randint(2, 3)
        return cls(init_num_nodes, add_num_edges)

#### GraphGameGUI Class
It contains the graphical user interface of the game implemented with tkinter.
<br><br>
Major contributor: Artem, Tom

In [16]:
class GraphGameGUI(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title('Graph Game')
        self.geometry('800x600')
        self['bg'] = 'white'
        # Make the window resizable
        self.resizable(False, False)  

        # Initialize the database
        initialize_database()

        # Create a variable for storing the current player and their score
        self.current_player = None
        self.current_balance = None

        # Initialize pygame mixer for music
        pygame.mixer.init()
        self.soundtrack_path = 'audio/Menu_Audio.mp3'
        pygame.mixer.music.load(self.soundtrack_path)
        # Change the voulme of music
        pygame.mixer.music.set_volume(0.5)
        # Loop
        pygame.mixer.music.play(-1)

        self.soundtrack_state = tk.BooleanVar(value=True)

        # Create a frame dict
        self.frames = {
            'menu': MainMenu(self),
            'play': Play(self),
            'win': Win(self),
            'lose': Lose(self),
            'login': Login(self),
            'register': Register(self),
            'leaderboard' : Leaderboard(self),
            'history' : PlayerHistory(self)
        }

        # Show main menu frame
        self.frames['login'].pack(fill='both', expand=True)

    def switch_soundtrack(self):
        if self.soundtrack_state.get():
            pygame.mixer.music.unpause()
        else:
            pygame.mixer.music.pause()

    def switch_frame(self, current_frame, new_frame):
        # Hide the current frame
        self.frames[current_frame].pack_forget()
        # Minimize the window
        self.geometry('1x1')
        # Update the window
        self.update_idletasks()
        # Restore the window size
        self.geometry('800x600')
        # Show the main menu frame
        self.frames[new_frame].pack(fill='both', expand=True)
        

class Login(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.configure(bg="white")
        
        title = tk.Label(self, text="Graph Game", font=('Helvetica', 60), bg='white')
        title.place(relx=0.5, rely=0.12, anchor='center')

        self.username_Label = tk.Label(self, text="Username:", font='Helvetica 30', bg='white')
        self.username_Label.place(relx=0.2, rely=0.3, anchor='center')
        self.username_Entry = tk.Entry(self, font='Helvetica, 30', width=15, bg='white')
        self.username_Entry.place(relx=0.5, rely=0.3, anchor='center')

        self.password_Label = tk.Label(self, text="Password:", font='Helvetica 30', bg='white')
        self.password_Label.place(relx=0.2, rely=0.4, anchor='center')
        self.password_Entry = tk.Entry(self, font='Helvetica, 30', width=15, bg='white', show='*')
        self.password_Entry.place(relx=0.5, rely=0.4, anchor='center')
        
        self.register_Button = tk.Button(self, text='Register', font='Helvetica 30', bg='white', command=lambda: self.parent.switch_frame('login', 'register'))
        self.register_Button.place(relx=0.5, rely=0.5, anchor='center')

        self.login_Button = tk.Button(self, text='Login', font='Helvetica 30', bg='white', command=self.login)
        self.login_Button.place(relx=0.5, rely=0.6, anchor='center')

    def login(self):
        username = self.username_Entry.get()    
        password = self.password_Entry.get()
        user = authenticate(username, password)
        if user:
            name, balance = user
            self.parent.current_player = name
            self.parent.current_balance = balance
            self.parent.frames['history'].load_player_history()
            self.parent.frames['play'].update_max_bid()
            self.parent.switch_frame('login', 'menu')
        else:
            messagebox.showinfo("Error", "The player is not found.")


class Register(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.configure(bg="white")
        
        title = tk.Label(self, text="Register", font=('Helvetica', 60), bg='white')
        title.place(relx=0.5, rely=0.12, anchor='center')

        self.username_Label = tk.Label(self, text="Username:", font='Helvetica 30', bg='white')
        self.username_Label.place(relx=0.2, rely=0.3, anchor='center')
        self.username_Entry = tk.Entry(self, font='Helvetica, 30', width=15, bg='white')
        self.username_Entry.place(relx=0.5, rely=0.3, anchor='center')

        self.password_Label = tk.Label(self, text="Password:", font='Helvetica 30', bg='white')
        self.password_Label.place(relx=0.2, rely=0.4, anchor='center')
        self.password_Entry = tk.Entry(self, font='Helvetica, 30', width=15, bg='white', show='*')
        self.password_Entry.place(relx=0.5, rely=0.4, anchor='center')
        
        self.password_repeat_Label = tk.Label(self, text="Repeat:", font='Helvetica 30', bg='white')
        self.password_repeat_Label.place(relx=0.22, rely=0.5, anchor='center')
        self.password_repeat_Entry = tk.Entry(self, font='Helvetica, 30', width=15, bg='white', show='*')
        self.password_repeat_Entry.place(relx=0.5, rely=0.5, anchor='center')

        self.register_Button = tk.Button(self, text='Register', font='Helvetica 30', bg='white', command=self.register)
        self.register_Button.place(relx=0.5, rely=0.6, anchor='center')

        self.login_Button = tk.Button(self, text='Login', font='Helvetica 30', bg='white', command=lambda: parent.switch_frame('register', 'login'))
        self.login_Button.pack(pady=(0,160),side=tk.BOTTOM)
        
    def register(self):
        username = self.username_Entry.get()
        password = self.password_Entry.get()
        password_repeat = self.password_repeat_Entry.get()
        if not username or not password or not password_repeat:
            messagebox.showinfo("Error", "Please fill in all the information.")
            return
        if password == password_repeat:
            if registered(username):
                messagebox.showinfo("Error", "The account has been registered.")
                return 
            register_player(username, password, 100)
            self.parent.current_player = username
            self.parent.current_balance = 100
            self.parent.frames['leaderboard'].search_engine.fetch_all_users_to_trie()
            self.parent.frames['leaderboard'].update_all_players()
            self.parent.switch_frame('register', 'menu')
        else:
            messagebox.showinfo("Error", "Passwords do not match.")

        
class MainMenu(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.configure(bg="white")
        self.parent = parent

        # Title
        title = tk.Label(self, text="Graph Game", font=('Helvetica', 60), bg='white')
        title.pack(pady=(50,20),side=tk.TOP)

        # Play button
        play_button = tk.Button(self, text="Play", command=lambda: self.parent.switch_frame('menu', 'play'),
                                bg='white', fg='black', font='Helvetica, 40', width=10)
        play_button.pack(pady = 10, side = tk.TOP)


        # How To Play Button
        history_Button = tk.Button(self, text='History', bg='white', fg='black',
                                   font='Helvetica, 40', width=10,
                                   command=lambda: self.parent.switch_frame('menu', 'history'))
        history_Button.pack(pady= 10)

        # Leaderboard Button
        leaderboard_button = tk.Button(self, text='Leaderboard', bg='white', fg='black',
                                       font='Helvetica, 40', width=10,
                                       command=lambda: self.parent.switch_frame('menu', 'leaderboard'))
        leaderboard_button.pack(pady=10)

        # Quit Button
        logout_button = tk.Button(self, text='Logout', command=lambda: self.parent.switch_frame('menu', 'login'),
                                  bg='white', fg='black', font='Helvetica, 40', width=10)
        logout_button.pack(pady=10)


class Leaderboard(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.search_engine = SearchEngine()
        self.players = self.search_engine.get_leaders(100)  # Get all players initially

        # Set the background color to white
        self.configure(bg="white")

        # Title
        leaderboard_label = tk.Label(self, text="Leaderboard", font=('Helvetica', 60), bg='white')
        leaderboard_label.place(relx=0.5, rely=0.12, anchor='center')

        # Put the back arrow image for the button
        self.back_image = tk.PhotoImage(file="images/back_arrow.png", master=self)
        # Make the image 10 times smaller
        self.resized_back_image = self.back_image.subsample(10, 10) 
        # Back Button 
        self.back_button = tk.Button(self, command=lambda: self.parent.switch_frame('leaderboard', 'menu'),
                             image=self.resized_back_image, background="white")
        self.back_button.pack(side="top", anchor="nw", padx=10, pady=10)

        self.words_for_autocompletion = []
        # Search entry combobox
        self.search_entry = ttk.Combobox(self, values=self.words_for_autocompletion) 
        self.search_entry.bind("<FocusIn>", self.text_search_if_empty)  
        self.search_entry.bind("<FocusOut>", self.text_search_if_empty_focus_out)
        # Bind the command to the entry if the key is released 
        self.search_entry.bind("<KeyRelease>", self.searching)
        self.search_entry.pack(side="top", padx=20, pady=(40, 0))  
        # Leaderboard Table: 

        # Create a treeview widget for the table
        self.tree = ttk.Treeview(self, columns=("Place", "Name", "Score"), show="headings")
        self.tree.pack(side="top", pady=(30, 0))

        # Add Column headings
        self.tree.heading("Place", text="Place", anchor=tk.CENTER)
        self.tree.heading("Name", text="Name", anchor=tk.CENTER)
        self.tree.heading("Score", text="Score", anchor=tk.CENTER)

        # Change the width of rows
        self.tree.column("Place", width=100, anchor=tk.CENTER)
        self.tree.column("Name", width=400, anchor=tk.CENTER)
        self.tree.column("Score", width=150, anchor=tk.CENTER)

        # Set font size and row height for the table
        self.style = ttk.Style()
        self.style.configure("Treeview", font=('Helvetica', 30), rowheight=40)
        self.style.configure("Treeview.Heading", font=('Helvetica', 30),rowheight=40)
        self.style.configure("Treeview.Row", font=('Helvetica', 30), rowheight=40)

        # Add scrollbars if there are too much columns
        self.y_scrollbar = ttk.Scrollbar(self, orient="vertical", command=self.tree.yview)
        self.y_scrollbar.pack(side="right", fill="y")
        self.x_scrollbar = ttk.Scrollbar(self, orient="horizontal", command=self.tree.xview)
        self.x_scrollbar.pack(side="bottom", fill="x")

        # Set scrollbars to the table
        self.tree.configure(yscrollcommand=self.y_scrollbar.set, xscrollcommand=self.x_scrollbar.set)

        # Call the function to set the values in the table
        self.update_treeview()

    def update_all_players(self):
        self.players = self.search_engine.get_leaders(100)
        self.update_treeview()

    def update_treeview(self):
        # Delete all previous data in the treeview
        self.tree.delete(*self.tree.get_children())
        # Populate the treeview with all players initially
        for i, row in enumerate(self.players):
            self.tree.insert("", "end", values=(i + 1, *row), tags=("Treeview.Row", "Treeview", "Treeview.Heading"))

    def text_search_if_empty(self, event):
        if self.search_entry.get() == "Search":
            self.search_entry.delete(0, tk.END)

    def text_search_if_empty_focus_out(self, event):
        if not self.search_entry.get():
            self.search_entry.insert(0, "Search")

    def searching(self, event):
        # Set the search text to lower letters for a more comfortabel search
        #search_text = self.search_entry.get().strip().lower()
        search_text = self.search_entry.get().strip()
        # Search for the list of words for autocompletion
        self.words_for_autocompletion = self.search_engine.complete_search(search_text)
        self.search_entry.config(values=self.words_for_autocompletion[:5])
        # If the search is empty, show all players 
        if not search_text or search_text == "search": 
            self.players = self.search_engine.get_leaders(100)
        else:
            # Show the only players whose name starts with the letters in the search_entry
            #self.players = [player for player in self.search_engine.get_leaders(100) if player[0].lower().startswith(search_text)]
            self.players = self.search_engine.search_results(search_text)
        # Update the table to see changes
        self.update_treeview()


class PlayerHistory(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent

        # Set the background color to white        
        self.configure(bg="white")
        
        # Put the back arrow image for the button
        self.back_image = tk.PhotoImage(file="images/back_arrow.png", master=self)
        # Make the image 10 times smaller
        self.resized_back_image = self.back_image.subsample(10, 10) 
        # Back Button 
        self.back_button = tk.Button(self, command=lambda: self.parent.switch_frame('history', 'menu'),
                             image=self.resized_back_image, background="white")
        self.back_button.pack(side="top", anchor="nw", padx=10, pady=10)
        # History Label
        self.history_label = tk.Label(self, text="History", font="Helvetica 60", background="white")
        self.history_label.place(relx=0.5, rely=0.05, anchor='center')
    
        # Create the treeview for storing the history of the player
        self.history_tree = ttk.Treeview(self, columns=("Bid", "Start Node", "End Node", "Outcome", "Score", "Date"), 
                                         show="headings")
        
        # Set column headings
        self.history_tree.heading("Bid", text="Bid")
        self.history_tree.heading("Start Node", text="Starting Node")
        self.history_tree.heading("End Node", text="Ending Node")
        self.history_tree.heading("Outcome", text="Outcome")
        self.history_tree.heading("Score", text="Score")
        self.history_tree.heading("Date", text="Date")
        
        # Set column widths
        self.history_tree.column("Bid", width=40)
        self.history_tree.column("Start Node", width=100)
        self.history_tree.column("End Node", width=80)
        self.history_tree.column("Outcome", width=50)
        self.history_tree.column("Score", width=20)
        self.history_tree.column("Date", width=150)
        
        # Change the font size 
        self.style = ttk.Style()
        self.style.configure("Treeview", font=('Helvetica', 20))
        self.style.configure("Treeview.Heading", font=('Helvetica', 20))
        self.style.configure("Treeview.Row", font=('Helvetica', 20))

        # Add scrollbar
        self.history_tree_scrollbar = ttk.Scrollbar(self, orient="vertical", command=self.history_tree.yview)
        self.history_tree.configure(yscrollcommand=self.history_tree_scrollbar.set)
        
        self.history_tree.pack(side="top", padx=10, pady=10, fill="both", expand=True)
        self.history_tree_scrollbar.pack(side="right", fill="y")

    def load_player_history(self):
        # Clear existing data
        for item in self.history_tree.get_children():
            self.history_tree.delete(item)
        
        # Get player history
        history = get_player_history(self.parent.current_player) 
        if history:
            # Insert history into treeview
            for bid, start, end, outcome, score, date in history:
                self.history_tree.insert("", "end", values=(bid, start, end, outcome, score, date))
    
    
class Play(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.parent = parent
        self.game = GraphGame.random_start()

        # The variable for new game
        self.game_started = True

        self.configure(bg="white")

        # generated_distance_variable for putting into Label
        self.generated_distance_variable = tk.StringVar()
        self.generated_distance_variable.set("Generated distance: -")

        # Put the back arrow image for the button
        self.back_image = tk.PhotoImage(file="images/back_arrow.png", master=self)
        # Make the image 10 times smaller
        self.resized_back_image = self.back_image.subsample(10, 10) 
        # Back Button 
        self.back_button = tk.Button(self, command=lambda: parent.switch_frame('play', 'menu'),
                             image=self.resized_back_image, background="white")
        self.back_button.pack(side="top", anchor="nw", padx=10, pady=10)

        #Label with the balance and balance wariable
        self.balance_label = tk.Label(self, text='', bg='white', fg='black', font='Helvetica, 20')
        self.balance_label.place(relx=0.8, rely=0.05, anchor='center')
        self.update_balance_label()

        # Label Bid before the scale
        self.bid_label = tk.Label(self, text = "Bid:", bg='white', fg='black',
                             font='Helvetica, 20')
        self.bid_label.place(relx=0.76, rely=0.35, anchor='center')

        # Scale with changing the bid
        self.bid_scale = tk.Scale(self, from_=0, to=self.parent.current_balance, orient=tk.HORIZONTAL,
                                  length=160,
                                  bg='white', fg='black',
                                  font = 'Helvetica, 20',
                                  )
        self.bid_scale.place(relx=0.89, rely=0.33, anchor='center')
         # Bind the method update_bid_scale_combobox_state to the combobox
        self.bid_scale.bind("<ButtonRelease-1>", self.update_bid_scale_combobox_state)

        # Starting node combobox
        self.starting_node_combobox = ttk.Combobox(self, width=5, state='disabled')
        # Fill the combobox with the values of the graph
        self.starting_node_combobox['values'] = self.game.get_nodes()
        self.starting_node_combobox.place(relx=0.94, rely=0.46, anchor='center')
        # Bind the method update_starting_ending_node_combobox_state to the combobox
        self.starting_node_combobox.bind("<<ComboboxSelected>>", self.update_starting_ending_node_combobox_state)
        # Bind the same method but as Focus Out to always check if the starting node combobox value was not deleted
        self.starting_node_combobox.bind("<FocusOut>", self.update_starting_ending_node_combobox_state)

        # Starting node combobox label
        self.starting_node_combobox_label = tk.Label(self, text="Starting node:", bg='white', fg='black',
                                    font='Helvetica, 20')
        self.starting_node_combobox_label.place(relx=0.81, rely=0.46, anchor='center')

        # Ending node combobox
        self.ending_node_combobox = ttk.Combobox(self, width=5, state='disabled')
        # Fill the combobox with the values of the graph
        self.ending_node_combobox['values'] = self.game.get_nodes()
        self.ending_node_combobox.place(relx=0.94, rely=0.57, anchor='center')

        # Ending node combobox label
        self.Ending_node_combobox_Label = tk.Label(self, text="Ending node:", bg='white', fg='black',
                                    font='Helvetica, 20')
        self.Ending_node_combobox_Label.place(relx=0.81, rely=0.57, anchor='center')

        # generated_distance_label with coefficient variable
        self.generated_distance_label = tk.Label(self, textvariable=self.generated_distance_variable, bg='white', font='Helvetica, 20')
        self.generated_distance_label.place(relx=0.86, rely=0.68, anchor='center')

        self.bet_button = tk.Button(self, text = "BET", bg = 'white', fg = 'black',
                               font='Helvetica, 25',
                               width=10,
                               command=self.bet_start_game)
        self.bet_button.pack(padx = (550,0), pady= (0,30), side = tk.BOTTOM)

        self.canvas = tk.Canvas(self, width=530, height=480, bg="white")
        self.canvas.place(relx=0.34, rely=0.59, anchor='center')

        self.update_plot()

    def update_balance_label(self):
        self.balance_label.config(text="Balance: " + str(self.parent.current_balance))
        self.balance_label.after(1000, self.update_balance_label)

    def update_bid_scale_combobox_state(self, event=None):
        # Check if bid_scale node combobox is empty
        if not self.bid_scale.get():  
            # Delete starting node selection
            self.starting_node_combobox.set('')  
            # Disable the combobox
            self.starting_node_combobox['state'] = 'disabled'  
        else:
            # Enable starting_node combobox
            self.starting_node_combobox['state'] = 'normal'  
            # Disable bid_scale combobox
            self.bid_scale.configure(state='disabled')

            # Set the base score for the score generator
            base_score = self.bid_scale.get()
            self.game.set_base_score(int(base_score))
    
    def update_starting_ending_node_combobox_state(self, event=None):
        """Method makes ending node combobox disabled if starting node combobox is not chosen"""
        
        # Check if starting node combobox is empty
        if not self.starting_node_combobox.get():  
            # Delete ending node selection
            self.ending_node_combobox.set('')  
            # Disable the combobox
            self.ending_node_combobox['state'] = 'disabled'  
        else:
            # Enable ending_node combobox
            self.ending_node_combobox['state'] = 'normal'  
            # Disable starting_node combobox
            self.starting_node_combobox['state'] = 'disable' 

            # Set the starting node for the graph game
            starting_node = self.starting_node_combobox.get()
            self.game.set_starting_node(int(starting_node))

            # Update the plot with node scores
            self.game.generate_cutoff()
            self.update_plot(with_node_scores=True)

    def update_plot(self, with_node_scores=False, result=False):
        # Create a matplotlib figure
        self.fig = Figure(figsize=(3,3), dpi=200)
        self.ax = self.fig.add_subplot(111)

        # Drawing nodes of the graph
        nx.draw_networkx_nodes(self.game.G, self.game.node_position, ax=self.ax, node_size=150)
        nx.draw_networkx_labels(self.game.G, self.game.node_position, ax=self.ax, font_size=10)

        # Draw edges of the nodes and set the width of each edge to be proportional to its weight
        edge_width = list(nx.get_edge_attributes(self.game.G, 'weight').values())
        nx.draw_networkx_edges(self.game.G, self.game.node_position, alpha=0.5, ax=self.ax, width=edge_width)

        if with_node_scores:
            if not self.game.starting_node or not self.game.score_generator:
                raise NameError('The score_generator or the starting node have not been defined.')

            # Get the shortest distances from the starting node to all other nodes
            node_to_dist = self.game.shortest_path(starting_node=self.game.starting_node)

            # Map the distance to each node to its score using the score_generator
            for node, dist in node_to_dist.items():
                node_to_dist[node] = self.game.score_generator.calculate_score(dist)

            # Create score labels for the networkx graph visualization
            label_pos = {}
            for node, coords in self.game.node_position.items():
                # Set the label to be 0.135 units above the original node position
                label_pos[node] = (coords[0], coords[1] + 0.135)

            nx.draw_networkx_labels(self.game.G, 
                                    pos=label_pos,
                                    ax=self.ax,
                                    labels=node_to_dist,
                                    font_size=4, 
                                    font_weight='bold')
            
            # Expand the graph to accomodate the score labels
            self.ax.set_ylim(tuple(i*1.2 for i in self.ax.get_ylim()))
        
        if result:
            # Find the shortest distance between the starting node and the ending node
            shortest_dist = self.game.shortest_path(self.game.starting_node, self.game.ending_node)
        
            # Find all the nodes between the shortest path using the nx library
            path = nx.shortest_path(self.game.G, source=self.game.starting_node, target=self.game.ending_node)
            path_edges = list(zip(path,path[1:]))

            # Draw the edges between in green if the player wins, red otherwise
            if shortest_dist < self.game.cutoff_distance:
                nx.draw_networkx_edges(self.game.G, self.game.node_position, ax=self.ax, edgelist=path_edges, edge_color='g', width=3)
            else:
                nx.draw_networkx_edges(self.game.G, self.game.node_position, ax=self.ax, edgelist=path_edges, edge_color='r', width=3)

            edge_labels = nx.get_edge_attributes(self.game.G, 'weight')
            nx.draw_networkx_edge_labels(self.game.G, self.game.node_position, edge_labels, ax=self.ax, font_size=4)
            
        # Draw the figure on the canvas
        self.canvas = FigureCanvasTkAgg(self.fig, self)
        self.canvas.draw() 
        self.canvas.get_tk_widget().place(relx=0.33, rely=0.60, anchor='center', width=600, height=500)

    # Function to update the maximum value of the bid scale
    def update_max_bid(self):
        self.bid_scale.config(to=self.parent.current_balance)

    def bet_start_game(self):
        if self.game_started == True:
            # Create the variable that states if all parameters are selected
            all_inputs_valid = True
        
            # Get the value of the bid 
            bid_amount = self.bid_scale.get()

            # Check if the bid amount is higher that zero, if it is zero, paint the label red
            if bid_amount == 0:
                self.bid_label.config(fg='red')
                all_inputs_valid = False
            else:
                self.bid_label.config(fg='black')

            # Check if the starting noded was selected, if it not, paint the label red
            starting_node = self.starting_node_combobox.get()
            if (not starting_node 
                or self.starting_node_combobox.get() == self.ending_node_combobox):
                self.starting_node_combobox_label.config(fg='red')
                all_inputs_valid = False
            else:
                self.starting_node_combobox_label.config(fg='black')

            # Check if the ending node was selected and get its value
            ending_node = self.ending_node_combobox.get()

            # Check if the ending noded was selected, if it not, paint the label red
            if( not ending_node 
               or self.starting_node_combobox.get() == self.ending_node_combobox.get()):
                self.Ending_node_combobox_Label.config(fg='red')
                all_inputs_valid = False
            else:
                self.Ending_node_combobox_Label.config(fg='black')
                self.game.set_ending_node(int(ending_node))

            # If all inputs are valid -> proceed with the game logic
            if all_inputs_valid:
                score = self.game.get_player_score()

                if(self.game.check_player_wins() == True):
                    self.parent.frames['win'].amount_of_winning_variable.set("Amount of winning: " + str(score))
                    self.parent.switch_frame('play', 'win')
                else:
                    score*=-1
                    self.parent.frames['lose'].amount_of_lose_variable.set("You lost: " + str(score)) 
                    score*=-1
                    self.parent.switch_frame('play', 'lose')

            self.update_plot(result=True, with_node_scores=True)
            self.game_started = False

            self.bet_button.config(text="Play Again")
            self.bet_button.update()

            self.starting_node_combobox['state'] = 'disabled'
            self.ending_node_combobox['state'] = 'disabled'
            self.starting_node_combobox.set('') 
            self.ending_node_combobox.set('')
            self.bid_scale.set(0)
            self.bid_scale['state'] = 'disabled' 

            # Update the generated distance label of the game 
            self.generated_distance_variable.set('Generated distance: ' + str(self.game.cutoff_distance))
            
            self.parent.current_balance += score
            # Update player's balance in db ...
            update_balance(self.parent.current_player, self.parent.current_balance)

            # Update the leaderboard
            self.parent.frames['leaderboard'].searching(event=None)
        
            outcome = 'win' if self.game.check_player_wins() else 'loss'
            log_game(self.parent.current_player, int(bid_amount), int(starting_node), int(ending_node), outcome, score)

            self.parent.frames['history'].load_player_history()

        else:
            self.bet_button.config(text="Play Again")

            self.game = GraphGame.random_start()
            self.update_plot(result=False)  
            self.game_started = True

            self.starting_node_combobox['state'] = 'disabled'
            self.bid_scale['state'] = 'normal' 

            self.starting_node_combobox['values'] = self.game.get_nodes()
            self.ending_node_combobox['values'] = self.game.get_nodes()

            self.starting_node_combobox.set('') 
            self.ending_node_combobox.set('')  
            self.bid_scale.set(0)

            self.ending_node_combobox['state'] = 'disabled'

            self.bid_scale.set(0)  

            self.bet_button.config(text="BET")
            self.bet_button.update()

            # Update the generated distance label of the game 
            self.generated_distance_variable.set('Generated distance: -')        

            if(self.parent.current_balance < 1):
                tk.messagebox.showinfo(title="Broke", message="Your balance is less than 1, here are 50 extra score")
                update_balance(self.parent.current_player, 50)
                self.parent.current_balance = 50
            
            self.update_max_bid()


class Win(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.configure(bg="white")

        #Balance variable for putting into Label
        self.amount_of_winning_variable = tk.StringVar()
        self.amount_of_winning_variable.set("Amount of winning: " + str(self.amount_of_winning_variable))

        # Put the back arrow image for the button
        self.back_image = tk.PhotoImage(file="images/back_arrow.png", master=self)
        # Make the image 10 times smaller
        self.resized_back_image = self.back_image.subsample(10, 10) 
        # Back Button
        self.back_button = tk.Button(self, command=lambda: parent.switch_frame('win', 'play'),
                             image=self.resized_back_image, background="white")
        self.back_button.pack(side="top", anchor="nw", padx=10, pady=10)


        # Put the back arrow image for the button
        self.win_image = tk.PhotoImage(file="images/win_picture.png", master=self)
        # Make the image 10 times smaller
        self.win_image = self.win_image.subsample(3, 3) 
        
        # Place the picture as a Label
        self.win_label_picture = tk.Label(self, image=self.win_image)
        self.win_label_picture.place(relx= 0.5, rely = 0.5, anchor='center')  
    
        self.you_won_label = tk.Label(self, text="YOU WON!!!", font = 'Helvetica, 50', bg = 'white')
        self.you_won_label.place(relx=0.5, rely=0.1, anchor='center')
        
        self.amount_of_winning_label = tk.Label(self, textvariable=self.amount_of_winning_variable, 
                                                font='Helvetica, 30',
                                                bg = 'white')
        self.amount_of_winning_label.place(relx= 0.5, rely= 0.88, anchor='center')


class Lose(tk.Frame):
    def __init__(self, parent):
        super().__init__(parent)
        self.configure(bg="white")

        #Balance variable for putting into Label
        self.amount_of_lose_variable = tk.StringVar()
        self.amount_of_lose_variable.set("You lost: " + str(self.amount_of_lose_variable))

        # Put the back arrow image for the button
        self.back_image = tk.PhotoImage(file="images/back_arrow.png", master=self)
        # Make the image 10 times smaller
        self.resized_back_image = self.back_image.subsample(10, 10) 
        # Back Button
        self.back_button = tk.Button(self, command=lambda: parent.switch_frame('lose', 'play'),
                             image=self.resized_back_image, background="white")
        self.back_button.pack(side="top", anchor="nw", padx=10, pady=10)


        # Put the back arrow image for the button
        self.lose_image = tk.PhotoImage(file="images/Loose_picture.png", master=self)
        # Make the image 10 times smaller
        self.lose_image = self.lose_image.subsample(3, 3) 
        
        # Place the picture as a Label
        self.lose_label_picture = tk.Label(self, image=self.lose_image)
        self.lose_label_picture.place(relx= 0.5, rely = 0.5, anchor='center')  
    
        self.you_lost_label = tk.Label(self, text="you lost):", font = 'Helvetica, 50', bg = 'white')
        self.you_lost_label.place(relx=0.5, rely=0.1, anchor='center')
        
        self.amount_of_losing_label = tk.Label(self, textvariable=self.amount_of_lose_variable, 
                                                font='Helvetica, 30',
                                                bg = 'white')
        self.amount_of_losing_label.place(relx= 0.5, rely= 0.88, anchor='center')

## 3. Run all the unit tests
Unit tests are conducted using the unittest module <br><br>
All unit tests are created by Zifan

In [17]:
import unittest
from unittest.mock import MagicMock, patch

### __Testing data structures__

#### Test Node class

In [18]:
class TestNode(unittest.TestCase):
    def setUp(self):
        self.node1 = Node(1)
        self.node2 = Node(2)
        self.node3 = Node(3)

    def test_node_creation(self):
        """Tests whether the node is created correctly"""
        self.assertEqual(self.node1.idx, 1)
        self.assertEqual(self.node2.idx, 2)

    def test_add_edge(self):
        """Testing the neighbor addition function"""
        self.node1.add_neighbour(self.node2, 10)
        self.assertIn(self.node2, self.node1.neighbours)
        self.assertEqual(self.node1.neighbours[self.node2], 10)

        with self.assertRaises(TypeError):
            self.node1.add_neighbour("not_a_node", 5)
        with self.assertRaises(TypeError):
            self.node1.add_neighbour(self.node3,"not_a_number")

    def test_node_comparison(self):
        """Testing the node comparison function"""
        self.assertTrue(self.node1 < self.node2)
        self.assertFalse(self.node1 > self.node2)

    def test_get_index(self):
        """Testing gets the index of the node"""
        self.assertEqual(self.node1.get_index(), 1)

    def test_get_neighbours(self):
        """Testing gets the neighbours of the node"""
        self.node1.add_neighbour(self.node2, 10)
        self.node1.add_neighbour(self.node3, 5)
        expected_neighbours = [(self.node2, 10), (self.node3, 5)]
        actual_neighbours = list(self.node1.neighbours.items())
        self.assertEqual(sorted(actual_neighbours, key=lambda x: x[0].idx),
                         sorted(expected_neighbours, key=lambda x: x[0].idx))

#### Test MinHeap class

In [19]:
class TestHeap(unittest.TestCase):
    def setUp(self):
        """Set up test fixtures"""
        self.nodes = [Node(i) for i in range(10)]

    def test_init_heap(self):
        """Test initializing heap"""
        heap = MinHeap([(3, self.nodes[3]), (1, self.nodes[1]), (2, self.nodes[2])])
        self.assertEqual(heap.pop()[0],1)
        self.assertEqual(heap.pop()[0],2)
        self.assertEqual(heap.pop()[0],3)

    def test_push_pop(self):
        """Test push and pop to the heap"""
        heap = MinHeap()
        heap.push((2, self.nodes[2]))
        heap.push((1, self.nodes[1]))
        heap.push((3, self.nodes[3]))
        self.assertEqual(heap.pop()[0], 1)
        self.assertEqual(heap.pop()[0], 2)
        self.assertEqual(heap.pop()[0], 3)

    def test_heap_property(self):
        """Test heap property"""
        heap = MinHeap()
        values = [1,2,3,4,5,6,7,8,9]
        for value in values:
            heap.push((value, Node(value)))
        result = []
        while len(heap) > 0:
            result.append(heap.pop()[0])
        self.assertEqual(result, sorted(values))

    def test_top(self):
        """Test the top element of the heap"""
        heap = MinHeap([(2, self.nodes[2]), (1, self.nodes[1]), (3, self.nodes[3])])
        self.assertEqual(heap.top()[0], 1)

    def test_empty_heap(self):
        """Test empty heap"""
        heap = MinHeap()
        self.assertIsNone(heap.top())
        self.assertIsNone(heap.pop())

    def test_duplicate(self):
        """Test duplicates into heap"""
        heap = MinHeap([(2, self.nodes[0]),(2, self.nodes[1]), (2, self.nodes[2])])
        heap.push((2, self.nodes[4]))
        heap.push((1, self.nodes[3]))
        self.assertEqual(heap.top()[0],1)
        heap.pop()
        for _ in range(4):
            self.assertEqual(heap.top()[0],2)
            heap.pop()

    def test_mixed_heap(self):
        """Test mixed situation"""
        heap = MinHeap()
        values = [(2, self.nodes[2]), (1, self.nodes[1]), (3, self.nodes[3])]
        for val in values:
            heap.push(val)
        self.assertEqual(heap.pop()[0],1)
        heap.push((0, self.nodes[0]))
        self.assertEqual(heap.pop()[0],0)

#### Test RandomisedSet class

In [20]:
class TestRandomisedSet(unittest.TestCase):
    def setUp(self):
        """INITIALIZE a RandomisedSet object"""
        self.randomised_set = RandomisedSet()
        rand.seed(0)

    def test_initialization(self):
        """The edges and edge_to_idx of the RandomisedSet should be empty"""
        self.assertFalse(self.randomised_set.edges)
        self.assertFalse(self.randomised_set.edge_to_idx)

    def test_add_edge(self):
        """Add an edge to the RandomisedSet"""
        self.randomised_set.add_edge_to_set(1, 2)
        self.assertEqual(self.randomised_set.edges, [(1, 2)])
        self.assertEqual(self.randomised_set.edge_to_idx, {(1, 2): 0})

    def test_remove_edge(self):
        """Remove an edge from the RandomisedSet"""
        self.randomised_set.add_edge_to_set(1, 2)
        self.randomised_set.remove_edge_from_set(1, 2)
        self.assertNotIn((1, 2),self.randomised_set.edge_to_idx)
        self.assertFalse(self.randomised_set.edges)

    def test_add_duplicate_edges(self):
        """Can not add duplicate edges to the RandomisedSet"""
        self.randomised_set.add_edge_to_set(1, 2)
        self.randomised_set.add_edge_to_set(2, 1)
        self.assertEqual(len(self.randomised_set.edges), 1)

    def test_random_edge(self):
        """Random edge extraction should correctly remove the edge from the set."""
        edges_to_add = [(1, 2), (3, 4)]
        for edge in edges_to_add:
            self.randomised_set.add_edge_to_set(*edge)
        extracted_edges = self.randomised_set.get_random_edge()
        self.assertIn(extracted_edges, edges_to_add)
        self.assertEqual(len(self.randomised_set.edges), 1)
        self.assertNotIn(extracted_edges, self.randomised_set.edge_to_idx)

#### Test Trie class

In [21]:
class TestTrie(unittest.TestCase):
    def setUp(self):
        self.trie = Trie()
        words = ["apple", "hello", "goodbye", "warwick", "warw", "warwi", "warwic"]
        for word in words:
            self.trie.insert(word)

    def test_insert_and_find(self):
        """Test insert and find"""
        self.assertTrue(self.trie.find("apple"))
        self.assertFalse(self.trie.find("UK"))

    def test_complete(self):
        """Test the complete_function"""
        self.assertIn('warwick', self.trie.complete('warw'))
        self.assertNotIn('apple', self.trie.complete('warw'))

    def test_fizzy_search(self):
        """Test the fizzy_search function"""
        self.assertIn('warwi', self.trie.fizzy_search('warwi', 1))
        self.assertIn('warwic', self.trie.fizzy_search('warwi', 1))
        self.assertNotIn('warwick', self.trie.fizzy_search('warwi', 1))

#### Test Graph class

In [22]:
class TestGraph(unittest.TestCase):
    def setUp(self):
        """initialize graph object"""
        self.graph = Graph(init_num_nodes=10, add_num_edges=5, edge_sd=5, edge_mean=10)

    def test_graph_initialization(self):
        """Test graph initialization"""
        self.assertEqual(self.graph.num_nodes, 10)
        self.assertGreaterEqual(self.graph.num_edges, 9)

    def test_add_edge(self):
        """Test add edge and validate graph"""
        initial_edges = self.graph.num_edges
        self.graph.add_edge_to_graph(1, 3)
        self.assertEqual(self.graph.num_edges, initial_edges + 1)
        self.assertTrue((1, 3) in self.graph.G.edges() or (2, 3) in self.graph.G.edges())

    def test_add_edge_invalid_edge(self):
        """Test add edge with invalid"""
        with self.assertRaises(ValueError):
            self.graph.add_edge_to_graph(100, 200)

    def test_shortest_path(self):
        """Test shortest path algorithm"""
        path = self.graph.shortest_path(1, 2)
        self.assertIsInstance(path, int)
        self.assertGreater(path, 0)

    def test_random_nodes_edge(self):
        """Test random nodes and edges generator"""
        self.graph.generate_random_nodes(num=2)
        self.assertEqual(self.graph.num_nodes, 12)
        self.graph.generate_random_edges(num=1)
        self.assertGreaterEqual(self.graph.num_edges, 10)

    def test_graph_str_repr(self):
        """Test graph string representation"""
        graph_str = str(self.graph)
        self.assertIn('|E| =', graph_str)
        self.assertIn('|V| =', graph_str)
        self.assertIn('E.x̄ =', graph_str)
        self.assertIn('E.σ =', graph_str)

### __Testing additional features__

#### Test SearchEngine class

In [30]:
class TestSearchEngine(unittest.TestCase):
    def setUp(self):
        """Set up test fixtures"""
        self.search_engine = SearchEngine()
        self.search_engine.trie = MagicMock()
        self.search_engine.fetch_all_users_to_trie = MagicMock()

    def test_complete_search(self):
        """Test the complete search method"""
        self.search_engine.trie.complete = MagicMock(return_value=["Femi", "Femu", "Femo"])
        result = self.search_engine.complete_search("Fem")
        self.assertEqual(result, ["Femi", "Femu", "Femo"])
        self.search_engine.trie.complete.assert_called_with("Fem")

    @patch('graph_game.search_engine.search_engine.DatabaseConnection')
    def test_search_results(self, mock_db):
        """Test the search results method"""
        mock_conn = mock_db.return_value.__enter__.return_value
        mock_conn.cursor.return_value.execute.return_value.fetchone.return_value = ("Femi", 100)
        mock_conn.cursor.return_value.execute.return_value.fetchall.return_value = [("Femi", 100)]
        self.search_engine.trie.fizzy_search = MagicMock(return_value=["Femi"])

        result = self.search_engine.search_results("Femi")
        self.assertEqual(result, [("Femi", 100)])

    @patch('graph_game.search_engine.search_engine.DatabaseConnection')
    def test_get_leaders(self, mock_db):
        """Test the get_leaders method"""
        mock_conn = mock_db.return_value.__enter__.return_value
        mock_conn.cursor.return_value.execute.return_value.fetchall.return_value = [("Femi", 100), ("Tom", 90)]
        result = self.search_engine.get_leaders(2)
        self.assertEqual(result, [("Femi", 100), ("Tom", 90)])
        self.assertRaises(ValueError, self.search_engine.get_leaders, 0)

    def test_fetch_all_users_to_trie(self):
        """Test the fetch_all_users_to_trie method"""
        self.search_engine.trie = None
        self.trie = MagicMock()
        with patch('graph_game.search_engine.search_engine.DatabaseConnection') as mock_db:
            mock_conn = mock_db.return_value.__enter__.return_value
            mock_conn.cursor.return_value.execute.return_value.fetchall.return_value = [("Femi",), ("Tom",)]
            self.search_engine.fetch_all_users_to_trie()
            self.assertIsNotNone(self.search_engine)

### __Testing game logic__

#### Test RandomScoreGenerator class

In [24]:
class TestRandomScoreGenerator(unittest.TestCase):
    def setUp(self):
        """set up the test case"""
        self.generator = RandomScoreGenerator()

    def test_init(self):
        """Test the __init__ method."""
        self.assertEqual(self.generator.base_score, 100)
        self.assertIsNone(self.generator._mean)
        self.assertIsNone(self.generator._sd)
        self.assertIsNone(self.generator._generated_distance)

    def test_set_mean_raises_value_error(self):
        """Test set_mean method raises a ValueError"""
        with self.assertRaises(ValueError):
            self.generator.set_mean(-1)
        with self.assertRaises(ValueError):
            self.generator.set_mean(1.1)

    def test_set_mean(self):
        """Test set_mean"""
        self.generator.set_mean(1)
        self.assertEqual(self.generator._mean, 1)

    def test_set_sd_raises_value_error(self):
        """Test set_sd method raises a ValueError"""
        with self.assertRaises(ValueError):
            self.generator.set_sd(-1)
        with self.assertRaises(ValueError):
            self.generator.set_sd("10")

    def test_set_sd(self):
        """Test set_sd correctly sets the sd"""
        self.generator.set_sd(5.0)
        self.assertEqual(self.generator._sd, 5.0)

    @patch('numpy.random.normal')
    def test_generate_random_distance(self, mock_normal):
        """Test generate_random_distance"""
        mock_normal.return_value = 5
        self.generator.set_mean(10)
        self.generator.set_sd(2)
        distance = self.generator.generate_random_distance()
        self.assertEqual(distance, 5)

    def test_generate_random_distance_raises_value_error(self):
        """Test generate_random_distance raises ValueError"""
        with self.assertRaises(ValueError):
            self.generator.generate_random_distance()
        self.generator.set_mean(10)
        with self.assertRaises(ValueError):
            self.generator.generate_random_distance()

    @patch('numpy.random.normal')
    def test_calculate_score(self, mock_normal):
        """Test calculate_score calculates correctly."""
        mean = 10
        sd = 5
        self.generator.set_mean(mean)
        self.generator.set_sd(sd)
        mock_normal.return_value = 10
        '''calculate score'''
        dist = 20
        expected_score = self.generator.calculate_score(dist)
        '''check if score is correct'''
        norm_distribution = stats.norm(mean, sd)
        prob = 1 - norm_distribution.cdf(dist)
        manual_score = int(1 / prob * sd * 2 + self.generator.base_score)
        self.assertEqual(expected_score, manual_score)

    @patch('numpy.random.normal')
    def test_generate_random_edge(self, mock_normal):
        """Test generate_random_edge returns weights"""
        mock_normal.return_value = 5
        weight = RandomScoreGenerator.generate_random_edge(10, 2)
        self.assertEqual(weight, 5)

    def test_generate_random_edge_raises_value_error(self):
        """Test generate_random_edge raises ValueError"""
        with self.assertRaises(ValueError):
            RandomScoreGenerator.generate_random_edge(-10, 2)
        with self.assertRaises(ValueError):
            RandomScoreGenerator.generate_random_edge(10, -2)
        with self.assertRaises(ValueError):
            RandomScoreGenerator.generate_random_edge(10, "2")

#### Test GraphGame class

In [25]:
class TestGraphGame(unittest.TestCase):
    def setUp(self):
        """Setup for graph"""
        self.game = GraphGame.random_start()
        self.game.set_base_score(100)
        self.game.set_starting_node(1)
        self.game.set_ending_node(4)
        self.game.score_generator = RandomScoreGenerator(self.game.base_score)  # Ensure a generator is set

    def test_set_base_score(self):
        """Test the base score."""
        self.assertEqual(self.game.base_score, 100)
        with self.assertRaises(TypeError):
            self.game.set_base_score('one hundred')

    def test_nodes_existence(self):
        """Test if starting and ending nodes exist in the graph."""
        self.assertIn(self.game.starting_node, self.game.get_nodes())
        self.assertIn(self.game.ending_node, self.game.get_nodes())
        with self.assertRaises(ValueError):
            self.game.set_starting_node(100)  # Assuming node index 100 does not exist

    def test_game_outcome(self):
        """Test the game outcome method."""
        self.game.generate_cutoff()  # First generate the cutoff
        self.assertTrue(isinstance(self.game.check_player_wins(), bool))  # Check if the result is a boolean

    def test_player_score(self):
        """Test score calculation."""
        self.game.generate_cutoff()  # Ensure cutoff is generated
        score = self.game.get_player_score()
        self.assertIsInstance(score, int)  # The score should be an integer

    def test_random_start(self):
        """Test starting a random game."""
        random_game = GraphGame.random_start()
        self.assertIsInstance(random_game, GraphGame)

In [ ]:
unittest.main(argv=[''], verbosity=2, exit=False)

## 4. Start the app

In [ ]:
app = GraphGameGUI()
app.mainloop()